In [ ]:
# Procesa el dataframe para el machine learning.

import os

import pandas as pd
import numpy as np

In [ ]:
path = "/content/drive/MyDrive/Colab/Subsidio_electricidad/"

# Escenarios.
RCP = ["RCP4p5", "RCP8p5"]
fut = [2030, 2050, 2070]

# Tarifas.
tarifa = ["1", "1A", "1A", "1A", "1B",
    "1B", "1C", "1D", "1E", "1F"]

# Directorios.
dirs = ["Actual"] + RCP
path_data = [path + "data/" + x + "/data_" for x in dirs]
path_socio = path + "Socioeconomicos/"
path_cfe = path + "CFE/"
csv = ".csv"
socio = [ "Datos_demograficos.csv",
    "GLP_2010_2016.csv" ]
cfe = [ "Electrico_residencial_2010_2017.csv",
    "Porcentaje_mensual.csv"]

def tar(x):
    if   x == "1" : y = 0
    elif x == "1A": y = 1
    elif x == "1B": y = 2
    elif x == "1C": y = 3
    elif x == "1D": y = 4
    elif x == "1E": y = 5
    elif x == "1F": y = 6
    else          : y = 6
    return y

def tar_inv(y):
    if   y == 0: x = "1"
    elif y == 1: x = "1A"
    elif y == 2: x = "1B"
    elif y == 3: x = "1C"
    elif y == 4: x = "1D"
    elif y == 5: x = "1E"
    elif y == 6: x = "1F"
    return x

In [ ]:
# Datos actuales.

p = 0
n = 10

data = pd.read_csv(path_data[p] + "0" + csv
    ).sort_values( ["CVE_INEGI", "Año"]
    ).reset_index(drop = True)

# Agrega el precio de gas LP.
df_socio = pd.read_csv(path_socio + socio[1])
data.rename({"Año": "Year"}, axis = 1, inplace = True)
for i in data.itertuples():
    data.loc[i[0], "$GLP"] = df_socio.loc[
        df_socio["CVE_ENT"] == i.CVE_ENT, str(i.Year)].values[0]

# Agrega la población y el PIB.    
df_socio = pd.read_csv(path_socio + socio[0])
k = ["Poblacion", "PIB"]
for i in data.itertuples():
    col = [x + " " + str(i.Year) for x in k]
    data.loc[i[0], k ] = df_socio.loc[
        df_socio["CVE_INEGI"] == i.CVE_INEGI,
        col].values[0]

# Agrega el consumo y la cantidad de usuarios.
df_cfe = pd.read_csv(path_cfe + cfe[0])
df_cfe_2 = df_cfe.where(df_cfe["Tarifa"] != "DAC").dropna()
l = ["1*", "DAC"]
k = ["Consumo", "Usuarios"]
col = ( [k[0] + "_" + x for x in l]
    + [k[1] + "_" + x for x in l] )
for i in data.itertuples():
    # Tipo de tarifa.
    data.loc[i[0], "Tarifa"] = ( df_cfe_2
        .loc[df_cfe["CVE_INEGI"] == i.CVE_INEGI,
        "Tarifa"].array[0] )
    
    col_2 = [x + " " + str(i.Year) for x in k]

    data.loc[i[0], col] = df_cfe.loc[
        (df_cfe["CVE_INEGI"] == i.CVE_INEGI)
        & ( df_cfe["Tarifa"].str.contains(l[0], regex = True)
        | ( df_cfe["Tarifa"] == l[1] ) ),
        col_2].values.flatten(order = "F")

data.rename({"Year": "Año"}, axis = 1, inplace = True)

# PIB per cápita y densidad de población.
data["PCI"] = data["PIB"] / data["Poblacion"]
data["Densidad_poblacion"] = data["Poblacion"] / data["Area"]

# Crea un dato por cada mes.
data_2 = data.copy()
data = data.loc[data.index.repeat(12)]
data["Mes"] = list(range(1, 13)) * 2469 * 7
data.reset_index(drop = True, inplace = True)

# Agrega las variables procesadas para cada municipio.
for i in range(1, n):
    data_i = pd.read_csv(path_data[p] + str(i) + csv
        ).sort_values( ["CVE_INEGI", "Año", "Mes"]
        ).reset_index( drop = True )
    data.iloc[:, i + 11] = data_i.iloc[:,i + 11]
    
    # Para la temperatura media, copia las variables adicionales.
    if i == 3:
        data.loc[ :, ["Tmean_max_2", "M_verano"] ] = (
          data_i.loc[ :, ["Tmean_max_2", "M_verano"] ] )

# Corrige los meses de inicio de
# verano no contemplados por CFE.
data["M_verano"] = data["M_verano"].astype(int)
data["M_verano"].clip(2, 5, inplace = True)

# Guardamos el archivo anual.
data_dummy = data.groupby( [ "CVE_INEGI", "Año" ] 
    ).mean().reset_index()
data_2.iloc[:,12:15] = data_dummy.iloc[:,10:13]
data_2.iloc[:,15:-10] = data_dummy.iloc[:,13:-10]
data_2.iloc[:,-10:-4] = data_dummy.iloc[:,-10:-4]
data_2 [ ["NOM_MUN", "NOM_ENT", "Tarifa"] ] = data.loc[
    data["Mes"] == 1, ["NOM_MUN", "NOM_ENT", "Tarifa"] ].values
data_2 = data_2[ data.columns.drop( "Mes" ) ]
data_2.to_csv(path_data[p][:-1] +
    "_yearly" + csv, index = False)

# Carga la información con el consumo mensual.
df_cfe = pd.read_csv(path_cfe + cfe[1])

# Calcula el consumo mensual.
for i in data.itertuples():
    data.loc[i[0], ["Consumo_1*", "Consumo_DAC"]] = (
        np.array( [ i[6], i[8] ] ) * df_cfe.loc[
        i.CVE_ENT, f"{i[-2]}-{i[-1]:02d}-01 00:00:00" ] )

# Calcula los meses correspondientes al verano.
data["Verano"] = np.nan
# 6 columnas por cada mes de verano.
m = ["M_" + str(x) for x in range(1, 7)]
# El número de mes correspondiente a cada mes de verano.
data[m[0]] = data["M_verano"] % 12
for i in range(len(m[1:])):
    data[m[i + 1]] = ( data[m[i]] + 1 ) % 12
data[m] = data[m].replace(0, 12)
# Cambiamos el nombre de las columnas de los meses de verano.
dict_m = dict(zip(m, ["Mes"] * 6))
data.rename(columns = dict_m, inplace = True)
# Si el mes es de verano, el valor de Verano es True.
for i in range(1, 7):
    data.iloc[:, -i] = data["Mes"].isin(data.iloc[:, -i])
data["Verano"] = data.iloc[:, -6:].any(axis = 1)
# Quitamos las columnas correspondientes a los meses de verano.
data = data.iloc[:, :-6]

# Los valores nulos de las variables son cero.
data = data.where(data.notna(), 0)
data["$luz"] = np.nan

# Guardamos el archivo mensual.
data.to_csv(path_data[p][:-1] 
    + csv, index = False)

# Calcula la tarifa real que deberían tener los municipios.
data_3 = data_2.copy()
data_3["Tarifa"] = ( ( data_3["Tmean_max_2"].apply(np.floor)
    .clip(24, 33) - 24 ).astype(int)
    .apply(lambda x: tarifa[x]) )
data_3.to_csv(path_data[p][:-1] +
    "_tarifa_teorica_yearly" + csv, index = False)
data_4 = data.copy()
data_4["Tarifa"] = data_3["Tarifa"].loc[
    data_2.index.repeat(12)
    ].reset_index(drop = True)
data_4.to_csv(path_data[p][:-1] +
    "_tarifa_teorica" + csv, index = False)

# Permite una tarifa diferente por cada año.
# Calcula la tarifa real que deberían tener los municipios.
#data_2 = data.copy()
#data["Tarifa"] = ( ( data["Tmean_max_2"].apply(np.floor)
#    .clip(24, 33) - 24 ).astype(int)
#    .apply(lambda x: tar[x]) )
#data.to_csv(path_data[p][:-1] +
#    "_tarifa_teorica" + csv, index = False)
#data_dummy = data.groupby( [ "CVE_INEGI", "Año" ] 
#    ).mean().reset_index()
#data_2.iloc[:,12:15] = data_dummy.iloc[:,12:15]
#data_2.iloc[:,15:-10] = data_dummy.iloc[:,15:-10]
#data_2.iloc[:,-10:-4] = data_dummy.iloc[:,-10:-4]
#data_2 [ ["NOM_MUN", "NOM_ENT"] ] = data.loc[
#    data["Mes"] == 1, ["NOM_MUN", "NOM_ENT"] ].values
#data_2 = data_2[ data.columns.drop( "Mes" ) ]
#data_2.to_csv(path_data[p][:-1] +
#    "_tarifa_teorica_yearly" + csv, index = False)

data

,CVE_INEGI,CVE_ENT,CVE_MUN,NOM_ENT,NOM_MUN,Consumo_1*,Usuarios_1*,Consumo_DAC,Usuarios_DAC,Tarifa,...,Tmean_max_2,M_verano,Densidad_poblacion,PCI,$luz,$GLP,Area,Año,Mes,Verano
0,1001,1,1,Aguascalientes,Aguascalientes,2.135152e+07,231727.0,1.619597e+06,4251.0,1,...,24.024788,5,6.833315,146798.659992,NaN,10.15100,116635.924759,2010,1,False
1,1001,1,1,Aguascalientes,Aguascalientes,1.959487e+07,231727.0,1.486348e+06,4251.0,1,...,24.024788,5,6.833315,146798.659992,NaN,10.15100,116635.924759,2010,2,False
2,1001,1,1,Aguascalientes,Aguascalientes,2.210239e+07,231727.0,1.676553e+06,4251.0,1,...,24.024788,5,6.833315,146798.659992,NaN,10.15100,116635.924759,2010,3,False
3,1001,1,1,Aguascalientes,Aguascalientes,2.408999e+07,231727.0,1.827321e+06,4251.0,1,...,24.024788,5,6.833315,146798.659992,NaN,10.15100,116635.924759,2010,4,False
4,1001,1,1,Aguascalientes,Aguascalientes,2.487438e+07,231727.0,1.886819e+06,4251.0,1,...,24.024788,5,6.833315,146798.659992,NaN,10.15100,116635.924759,2010,5,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207391,32058,32,58,Zacatecas,Santa MarÃ­a de la Paz,6.989791e+04,1119.0,6.708718e+02,2.0,1,...,23.473484,4,0.095151,83642.721273,NaN,11.91534,27566.664781,2016,8,True
207392,32058,32,58,Zacatecas,Santa MarÃ­a de la Paz,6.147558e+04,1119.0,5.900353e+02,2.0,1,...,23.473484,4,0.095151,83642.721273,NaN,11.91534,27566.664781,2016,9,True
207393,32058,32,58,Zacatecas,Santa MarÃ­a de la Paz,6.399825e+04,1119.0,6.142476e+02,2.0,1,...,23.473484,4,0.095151,83642.721273,NaN,11.91534,27566.664781,2016,10,False
207394,32058,32,58,Zacatecas,Santa MarÃ­a de la Paz,5.942151e+04,1119.0,5.703206e+02,2.0,1,...,23.473484,4,0.095151,83642.721273,NaN,11.91534,27566.664781,2016,11,False


In [ ]:
# Archivo actual.
data_0 = pd.read_csv( path_data[0][:-1] + csv )
data = data_0[ data_0["Año"] == 2010 ].sort_values(
    ["CVE_INEGI", "Mes"] ).reset_index()
data = data[ data_0.columns ].drop( columns = "Año" )

# Iteramos para cada RCP.
for p in range(1, 3):

    # Iteramos para cada periodo futuro.
    for f in fut:

            for i in [10, 11]:
                # Calculamos el crecimiento de población y de PIB futuro.
                if not os.path.exists( path_data[p] + str(i)
                    + "." + str(f) + ".original" + csv ):

                    # Modelo, periodo actual.
                    data_i_0 = pd.read_csv(path_data[p] +
                        str(i) + ".2010" + csv).sort_values(
                        ["CVE_INEGI", "Mes"] ).reset_index()
                    data_i_0 = data_i_0[ data.columns.drop("Verano") ]
                    # Modelo, periodo futuro.
                    data_i = pd.read_csv(path_data[p] +
                        str(i) + "." + str(f) + csv).sort_values(
                        ["CVE_INEGI", "Mes"] ).reset_index()
                    data_i = data_i[ data.columns.drop("Verano") ]
                    # Guardamos una copia del archivo original.
                    data_i.to_csv(path_data[p] + str(i) + "." +
                         str(f) + ".original" + csv, index = False)

                    # Porcentaje de cambio.
                    data_i.iloc[:,i + 11] /= data_i_0.iloc[:,i + 11]
                    data_i.to_csv(path_data[p] + str(i) + 
                        "." + str(f) + "_perc" + csv, index = False)
                
                    # Valor proyectado.
                    data_i.iloc[:,i + 11] *= data.iloc[:,i + 11]
                    data_i.to_csv(path_data[p] + str(i) + 
                        "." + str(f) + csv, index = False)

In [ ]:
# Datos futuros.

# Iteramos para cada RCP.
for p in range(1, 3):
    
    # Iteramos para cada periodo futuro.
    for f in fut:

        data = pd.read_csv( path_data[p]
            + "0." + str(f) + csv
            ).sort_values( "CVE_INEGI"
            ).reset_index( drop = True )


        # Agrega el precio de gas LP.
        df_socio = pd.read_csv(path_socio + socio[1])
        for i in data.itertuples():
            data.loc[i[0], "$GLP"] = df_socio.loc[
                df_socio["CVE_ENT"] == i.CVE_ENT,
                str(2010) ].values[0]

        # Crea un dato por cada mes.
        data_2 = data.copy()
        data = data.loc[data.index.repeat(12)]
        data["Mes"] = list(range(1, 13)) * 2469
        data.reset_index(drop = True, inplace = True)

        # Agrega las variables procesadas para cada municipio.
        n = 12
        for i in range(1, n):
            data_i = pd.read_csv(path_data[p] +
                str(i) + "." + str(f) + csv ).sort_values(
                ["CVE_INEGI", "Mes"] ).reset_index( drop = True )
            data.iloc[:, i + 11 ] = data_i.iloc[:, i + 11 ]
            
            # Para la temperatura media, copia las variables adicionales.
            if i == 3:
                data.loc[ :, ["Tmean_max_2", "M_verano"] ] = (
                    data_i.loc[ :, ["Tmean_max_2", "M_verano"] ] )

        # PIB per cápita y densidad de población.
        data["PCI"] = data["PIB"] / data["Poblacion"]
        data["Densidad_poblacion"] = ( 
            data["Poblacion"] / data["Area"] )
            
        # Corrige los meses de inicio de
        # verano no contemplados por CFE.
        data["M_verano"] = data["M_verano"].astype(int)
        data["M_verano"].clip(2, 5, inplace = True)

        # Convierte la temperatura media máxima en el tipo de tarifa.
        data["Tarifa"] = ( ( data["Tmean_max_2"].apply(np.floor)
            .clip(24, 33) - 24 ).astype(int)
            .apply(lambda x: tarifa[x]) )
        
        data_temp = data.copy()
        
        # Calcula un archivo con la tarifa
        # real y otro con la teórica.
        for i in range(0, 3):
            
            # Corrige a la las tarifas para que sean
            # iguales o mayores a las reales actuales.
            if i == 0:
                data_0 = pd.read_csv( path_data[0][:-1] + csv )
                data_0 = data_0[ data_0["Año"] == 2010
                    ].sort_values( [ "CVE_INEGI", "Mes" ]
                    ).reset_index(drop = True)
                data_0["Tarifa"] = data_0["Tarifa"].apply(tar)
                data[  "Tarifa"] = data[  "Tarifa"].apply(tar)
                data[  "Tarifa"] = pd.concat( [ data["Tarifa"],
                    data_0["Tarifa"] ], axis = 1 ).max( axis = 1 )
                data[  "Tarifa"] = data[  "Tarifa"].apply(tar_inv)
            
            # Establece la temperatura actual con población y PIB futuro.
            elif i == 1:
                cols = ["Poblacion", "PIB", "PCI",
                    "Densidad_poblacion", "Tarifa"]
                data_0 = pd.read_csv( path_data[0][:-1] + csv )
                data_0 = data_0[ data_0["Año"] == 2010
                    ].sort_values( [ "CVE_INEGI", "Mes" ]
                    ).reset_index(drop = True)
                data_0[cols] = data[cols]
                data = data_0.copy().drop(columns = "Verano")
                # Agregamos el año correcto.
                data["Año"] = f

            else: data = data_temp.copy()

            # Guarda el archivo anual.
            data_dummy = data.groupby( [ "CVE_INEGI" ] 
                ).mean().reset_index()
            data_2.iloc[:,12:15] = data_dummy.iloc[:,-20:-17]
            data_2.iloc[:,15:-10] = data_dummy.iloc[:,-17:-11]
            data_2.iloc[:,-10:-4] = data_dummy.iloc[:,-11:-5]
            data_2 [ ["NOM_MUN", "NOM_ENT", "Tarifa"] ] = data.loc[
                data["Mes"] == 1, [ "NOM_MUN",
                "NOM_ENT", "Tarifa" ] ].values
            col = [x for x in data.columns]
            col.remove("Mes")
            data_2 = data_2[col]

            if i == 0:
                data_2.to_csv(path_data[p][:-1] + "." + str(f) 
                    + "_yearly" + csv, index = False)
            elif i == 1:
                data_2.to_csv(path_data[p][:-1] + "." + str(f) 
                    + "_pob_pib_yearly" + csv, index = False)
            elif i == 2 :
                data_2.to_csv(path_data[p][:-1] + "." + str(f) 
                    + "_tarifa_teorica_yearly" + csv, index = False)
            
            # Calcula los meses correspondientes al verano.
            data["Verano"] = np.nan
            # 6 columnas por cada mes de verano.
            m = ["M_" + str(x) for x in range(1, 7)]
            # El número de mes correspondiente a cada mes de verano.
            data[m[0]] = data["M_verano"] % 12
            for j in range(len(m[1:])):
                data[m[j + 1]] = ( data[m[j]] + 1 ) % 12
            data[m] = data[m].replace(0, 12)
            # Cambiamos el nombre de las
            # columnas de los meses de verano.
            dict_m = dict(zip(m, ["Mes"] * 6))
            data.rename(columns = dict_m, inplace = True)
            # Si el mes es de verano, el valor de Verano es True.
            for j in range(1, 7):
                data.iloc[:, -j] = data["Mes"].isin(data.iloc[:, -j])
            data["Verano"] = data.iloc[:, -6:].any(axis = 1)
            # Quitamos las columnas correspondientes
            # a los meses de verano.
            data = data.iloc[:, :-6]
            
            # Los valores nulos de las variables son cero.
            data  = data.where(data.notna(), 0)
            data["$luz"] = np.nan

            # Guardamos el archivo mensual.
            if i == 0:
                data.to_csv(path_data[p][:-1] + "."
                    + str(f) + csv, index = False)
            elif i == 1:
                data.to_csv(path_data[p][:-1] + "." + str(f) 
                    + "_pob_pib" + csv, index = False)
            elif i == 2 :
                data.to_csv(path_data[p][:-1] + "." + str(f) 
                    + "_tarifa_teorica" + csv, index = False)
            
pd.read_csv(path_data[p][:-1] + "." + str(f) 
    + "_tarifa_teorica" + csv)

,CVE_INEGI,CVE_ENT,CVE_MUN,NOM_ENT,NOM_MUN,Consumo_1*,Usuarios_1*,Consumo_DAC,Usuarios_DAC,Tarifa,...,Tmean_max_2,M_verano,Densidad_poblacion,PCI,$luz,$GLP,Area,Año,Mes,Verano
0,1001,1,1,Aguascalientes,Aguascalientes,0.0,0.0,0.0,0.0,1A,...,26.265261,4,6.605042,973428.567438,NaN,10.151,116635.924759,2070,1,False
1,1001,1,1,Aguascalientes,Aguascalientes,0.0,0.0,0.0,0.0,1A,...,26.265261,4,6.605042,973428.567438,NaN,10.151,116635.924759,2070,2,False
2,1001,1,1,Aguascalientes,Aguascalientes,0.0,0.0,0.0,0.0,1A,...,26.265261,4,6.605042,973428.567438,NaN,10.151,116635.924759,2070,3,False
3,1001,1,1,Aguascalientes,Aguascalientes,0.0,0.0,0.0,0.0,1A,...,26.265261,4,6.605042,973428.567438,NaN,10.151,116635.924759,2070,4,True
4,1001,1,1,Aguascalientes,Aguascalientes,0.0,0.0,0.0,0.0,1A,...,26.265261,4,6.605042,973428.567438,NaN,10.151,116635.924759,2070,5,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29623,32058,32,58,Zacatecas,Santa MarÃ­a de la Paz,0.0,0.0,0.0,0.0,1A,...,26.752193,4,0.139769,488998.846764,NaN,10.129,27566.664781,2070,8,True
29624,32058,32,58,Zacatecas,Santa MarÃ­a de la Paz,0.0,0.0,0.0,0.0,1A,...,26.752193,4,0.139769,488998.846764,NaN,10.129,27566.664781,2070,9,True
29625,32058,32,58,Zacatecas,Santa MarÃ­a de la Paz,0.0,0.0,0.0,0.0,1A,...,26.752193,4,0.139769,488998.846764,NaN,10.129,27566.664781,2070,10,False
29626,32058,32,58,Zacatecas,Santa MarÃ­a de la Paz,0.0,0.0,0.0,0.0,1A,...,26.752193,4,0.139769,488998.846764,NaN,10.129,27566.664781,2070,11,False
